In [2]:
from datasets import load_dataset
dataset = load_dataset("ashraq/esc50", split="train[0:10]", cache_dir="/Volumes/Bobo/System/.cache/huggingface/datasets/")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
audio_sample = dataset[0]
print(audio_sample)


{'filename': '1-100032-A-0.wav', 'fold': 1, 'target': 0, 'category': 'dog', 'esc10': True, 'src_file': 100032, 'take': 'A', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x304bacbd0>}


In [4]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],rate = audio_sample["audio"]["sampling_rate"])

In [5]:
from transformers import pipeline
zero_shot_classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")

Device set to use mps:0


In [6]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [7]:
audio_sample["audio"]["sampling_rate"]

44100

In [8]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=48_000))

In [9]:
audio_sample = dataset[0]
print(audio_sample)

{'filename': '1-100032-A-0.wav', 'fold': 1, 'target': 0, 'category': 'dog', 'esc10': True, 'src_file': 100032, 'take': 'A', 'audio': <datasets.features._torchcodec.AudioDecoder object at 0x38a184090>}


In [10]:
IPythonAudio(audio_sample["audio"]["array"],rate = audio_sample["audio"]["sampling_rate"])

In [11]:
candidate_labels=["Sound of a dog", "Sound of vacuum cleaner"]

In [12]:
zero_shot_classifier(audio_sample["audio"]["array"], candidate_labels=candidate_labels)

[{'score': 0.9996850490570068, 'label': 'Sound of a dog'},
 {'score': 0.00031492006382904947, 'label': 'Sound of vacuum cleaner'}]

In [13]:
#import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# 设置代理
# os.environ['http_proxy'] = 'http://localhost:15236'
# os.environ['https_proxy'] ='http://localhost:15236'

In [26]:
#dataset = load_dataset("librispeech_asr", split="train.clean.100", cache_dir="/Volumes/Bobo/System/.cache/huggingface/datasets/")
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", cache_dir="/Volumes/Bobo/System/.cache/huggingface/datasets/")
ds = load_dataset("distil-whisper/librispeech_asr_dummy", "clean",cache_dir="/Volumes/Bobo/System/.cache/huggingface/datasets/")

README.md: 0.00B [00:00, ?B/s]

clean/validation-00000-of-00001-538c39ec(…):   0%|          | 0.00/9.20M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

In [27]:
ds

DatasetDict({
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'whisper_transcript'],
        num_rows: 73
    })
})

In [37]:
example = dataset["validation"][0]

In [17]:
dataset_head = dataset
list(dataset_head)

['validation']

In [38]:
example

{'file': '1272-128104-0000.flac',
 'audio': <datasets.features._torchcodec.AudioDecoder at 0x31938f690>,
 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
 'speaker_id': 1272,
 'chapter_id': 128104,
 'id': '1272-128104-0000'}

In [ ]:
IPythonAudio(example["audio"]["array"],rate = audio_sample["audio"]["sampling_rate"])

In [40]:
asr = pipeline(task="automatic-speech-recognition", model="distil-whisper/distil-small.en")
#asr = pipeline("automatic-speech-recognition", model="distil-whisper/distil-small.en")

Device set to use mps:0


In [41]:
asr(example["audio"]["array"])

/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' Mr. Quilter is the Apostle of the Middle Classes, and we are glad to welcome his Gospel.'}

In [42]:
example["text"]

'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

In [43]:
import gradio as gr
demo = gr.Blocks()

In [44]:
def transcribe_speech(filepath):
    if filepath is None:
        gr.Warning("No audio found, please retry!")
        return ""
    output=asr(filepath)
    print(output)
    return output("text")
    

In [45]:
mic_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs = gr.Audio(sources="microphone",
                     type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                      lines=3),
    flagging_mode="never",
)

In [46]:
file_transcribe = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(sources="upload",
                   type="filepath"),
    outputs=gr.Textbox(label="Transcription",
                      lines=3),
    flagging_mode="never"
)

In [ ]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe,
        file_transcribe],
        ["Transcribe Microphone-test",
        "Transcribe Audio File"],
    )

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' Hello, how are you?'}


Traceback (most recent call last):
  File "/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/huggingface/lib/python3.11/site-packages/anyio/to_thread.p